 ## Step 1. Create a new SQLite database

In [1]:
import sqlite3

# Create (or connect to) the database
conn = sqlite3.connect("retail_customer_experience.db")

# Check connection
print("Database created successfully!")


Database created successfully!


## Step 2. Load CSV files into DataFrames

In [3]:
import pandas as pd

customers = pd.read_csv("../data/customers.csv")
transactions = pd.read_csv("../data/transactions.csv")
support_tickets = pd.read_csv("../data/support_tickets.csv")
campaigns = pd.read_csv("../data/campaigns.csv")
customer_reviews = pd.read_csv("../data/customer_reviews_complete.csv")
interactions = pd.read_csv("../data/interactions.csv")


In [4]:
customers.head()


,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,4c30e132-0704-4459-a509-9eddde934977,Mark Johnson,40.0,Male,mark.johnson@yahoo.com,989.608.3863,819 Johnson Course,Houston,Texas,29158.0,2024-04-25,NaN
1,68bec407-275f-4b5b-9a82-13d02f54626a,Robert Smith,33.0,Male,smithr@yahoo.com,(518)349-5931x0341,35116 Michael Key Suite 078,Austin,Texas,16862.0,2021-05-30,in-store
2,4466459f-76c8-433c-814e-6d59cb4131fc,Jamie Chavez,42.0,Female,jchavez@gmail.com,364.583.5030x564,419 Amanda Gardens,Detroit,Michigan,21918.0,2023-12-14,online
3,04c36a25-02f3-462c-92b0-6bf291c57706,Thomas Bradley,53.0,Male,thomas.bradley@hotmail.com,(332)887-1012x269,7242 Julie Plain Suite 969,Fort Worth,Texas,52851.0,2022-07-11,both
4,e916df3d-c3f5-40b0-8ae2-5d043be88300,Jane Ferrell,32.0,Female,jane.ferrell@hotmail.com,5484281489,845 Kelly Estate,Atlanta,Georgia,59971.0,2020-09-06,online


## Step 3. Write each DataFrame to the SQLite database

In [5]:
# Save each dataset as a table
customers.to_sql("customers", conn, if_exists="replace", index=False)
transactions.to_sql("transactions", conn, if_exists="replace", index=False)
support_tickets.to_sql("support_tickets", conn, if_exists="replace", index=False)
campaigns.to_sql("campaigns", conn, if_exists="replace", index=False)
customer_reviews.to_sql("customer_reviews", conn, if_exists="replace", index=False)
interactions.to_sql("interactions", conn, if_exists="replace", index=False)

print("✅ All tables created in SQLite database.")


✅ All tables created in SQLite database.


## Step 4. Verify tables exist

In [6]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


[('customers',), ('transactions',), ('support_tickets',), ('campaigns',), ('customer_reviews',), ('interactions',)]


## Step 5. (Optional) Inspect a sample join query

In [7]:
query = """
SELECT c.customer_id, c.full_name, c.city, c.gender,
       SUM(t.price * t.quantity) AS total_spent,
       COUNT(t.transaction_id) AS num_transactions
FROM customers c
LEFT JOIN transactions t ON c.customer_id = t.customer_id
GROUP BY c.customer_id
LIMIT 10;
"""
result = pd.read_sql_query(query, conn)
result


,customer_id,full_name,city,gender,total_spent,num_transactions
0,00012aa8-e99c-4e30-b3f6-1f7e36adc517,Ruth White,Vancouver,Female,2734.46,5
1,000feeed-f931-4908-b539-29ab57e595be,Makayla Hamilton,Durham,Female,2813.06,6
2,00107f62-0530-48ec-a56a-49d90944eafe,Jason Willis,Tacoma,Male,161.99,1
3,005ae094-2f68-4a6d-91f0-73b38b890692,Gary Mack,Rockford,Male,3691.46,6
4,00a6835b-c50a-49a9-a1b5-19d74d3e1863,William Sparks,Fort Worth,Male,3820.98,7
5,00a90d43-e62f-4e16-9a17-948d25c9684b,Tina Gilbert,Lansing,Female,5636.89,7
6,00ca767b-0217-46ba-b482-021916fa1be0,Andrew Adams,Houston,Male,220.59,2
7,00ca868d-4a6a-47c1-9231-a8279337da2f,Victoria Mcclain,San Francisco,Female,NaN,0
8,00df5d7c-c28c-436c-aa68-e3ceb775fff2,Alexander Lee,San Diego,Male,6560.97,5
9,01028498-4406-4014-a30c-2b9a605c2bfc,Andrew Evans,San Jose,Male,5507.84,6


## Step 6. Save and close connection

In [8]:
conn.commit()
conn.close()
print("Database saved and connection closed.")


Database saved and connection closed.
